In [1]:
import pandas as pd
import numpy as np
import folium 
import geopandas as gpd
import folium.plugins as plugins

from datetime import datetime


from difflib import get_close_matches

import matplotlib.pyplot as plt
import matplotlib.pyplot

%matplotlib inline

# import chart_studio.plotly as py
# import plotly.graph_objects as go

import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

from scipy.stats.mstats import winsorize

In [2]:
#load tech_1
tech1 = pd.read_csv(
    r'C:\Users\duchi\OneDrive - University of South Australia\Python\3 webscrapping\3 summary\tech_1.csv',
    encoding='iso-8859-1')

#load location tech1 (for non-similar names only (around 2000))
loc_tech1 = pd.read_csv(
    r'C:\Users\duchi\OneDrive - University of South Australia\Python\3 webscrapping\3 summary\loc_for_tech_1.csv',
    encoding='iso-8859-1')

print(len(loc_tech1))

5375


In [3]:
#prepare tech_2

df1 = pd.read_csv(
    r'C:\Users\duchi\OneDrive - University of South Australia\Python\3 webscrapping\4 do it again\spacy name entities\name_entity_proquest_tech(2).csv',
    encoding='iso-8859-1')
df1_1 = pd.DataFrame()

#prep the data for organization name
for i in range(len(df1)):

    x = df1['organization'].loc[i]
    list_1 = []

    if type(x) != float:
        list_1 = list_1 + [y.strip("''") for y in x.split(', ')]
        temp = pd.DataFrame({'org': list_1})
        temp['tech'] = df1.tech.loc[i]
        df1_1 = pd.concat([df1_1, temp], ignore_index=True)
    else:
        pass

test1 = pd.read_csv(
    r'C:\Users\duchi\OneDrive - University of South Australia\Python\3 webscrapping\2 newspaper\technews(2)_1000.csv',
    encoding='iso-8859-1')
test2 = pd.read_csv(
    r'C:\Users\duchi\OneDrive - University of South Australia\Python\3 webscrapping\2 newspaper\technews(2)_2000.csv',
    encoding='iso-8859-1')
test3 = pd.read_csv(
    r'C:\Users\duchi\OneDrive - University of South Australia\Python\3 webscrapping\2 newspaper\technews(2)_3000.csv',
    encoding='iso-8859-1')

pq_link = pd.concat([test1,test2,test3], ignore_index=True)

#merge date time with organization name
print(len(df1))
df1 = pd.merge(df1, pq_link[['tech','link','date_']], how='left', on=['tech','link'])
print(len(df1))

df1 = df1.reset_index()

3117
3117


In [4]:
# load location of organization (for non-similar name only)
loc1 = pd.read_csv(
    r'C:\Users\duchi\OneDrive - University of South Australia\Python\3 webscrapping\4 do it again\spacy name entities\geo_location(2)_1.csv',
    encoding='iso-8859-1')

print(len(loc1))

2874


In [5]:
# more location for organization (from similar name)
loc_similar1 = pd.read_csv(
    r'C:\Users\duchi\OneDrive - University of South Australia\Python\3 webscrapping\4 do it again\spacy name entities\geo_location_similar.csv',
    encoding='iso-8859-1')

print(len(loc_similar1))

14419


In [6]:
print(len(df1))
print(len(tech1))
df1 = pd.concat([df1,tech1],ignore_index=True)
print(len(df1))
df1 = df1.drop_duplicates(ignore_index=True)
print(len(df1))
df1.head()

3117
4373
7490
7490


,index,link,tech,person,nationality,building,organization,country,location,product,event,money,date_
0,0,https://www.proquest.com/anznews/docview/2673010738/5C465B1D225B42F9PQ/1?accountid=14649,start-up,"tom, tom s",NaN,NaN,"western ridge brewery, stien s taphouse","adelaide, nuriootpa",NaN,NaN,NaN,"an extra cents, the two million dollars","Jun 3, 2022"
1,1,https://www.proquest.com/anznews/docview/2677962027/5C465B1D225B42F9PQ/2?accountid=14649,start-up,"lynch, eddie mcguire, wrest, gill mclachlan, mclachlan, rockliff, gillon mclachlan, jeremy rockliff, jeff kennett, john olsen, david koch, colin carter, brett stubbs","tasmania, tasmanians, tasmanian, hawthorn",NaN,"collingwood, carbine club, afl, gws, hawks","tasmania, north melbourne, australia, hobart, melbourne, tassie, adelaide, port adelaide",NaN,NaN,NaN,NaN,"Jun 17, 2022"
2,2,https://www.proquest.com/anznews/docview/2678516294/5C465B1D225B42F9PQ/3?accountid=14649,start-up,"brendan dobson, hope forrest, noel lindsay, dobson",NaN,NaN,"the university of adelaide s, renmark university of adelaide s, university of adelaide, the renmark irrigation trust",NaN,"riverland, australia, the riverland region",NaN,NaN,NaN,"Jun 21, 2022"
3,3,https://www.proquest.com/anznews/docview/2702628411/5C465B1D225B42F9PQ/4?accountid=14649,start-up,"sam messina, christian alexandru, messina, alexandru, ortomi, noel lindsay, lindsay",NaN,NaN,"the university of adelaide s, ortomi, the university of adelaide",us,NaN,NaN,NaN,NaN,"Aug 17, 2022"
4,4,https://www.proquest.com/anznews/docview/2700310489/5C465B1D225B42F9PQ/5?accountid=14649,start-up,"richard payne, john lasich, payne","victorian, australian",NaN,"raygen, agl chevron s, equinor photon energy, schlumberger new energy, agl","carwarp, shanghai, russia, ukraine, liddell, nsw, south australia","europe, the netherlands",NaN,NaN,NaN,"Aug 9, 2022"


In [7]:
print(len(loc1))
print(len(loc_tech1))
print(len(loc_similar1))

loc = pd.concat([loc1,loc_tech1,loc_similar1],ignore_index=True)

print(len(loc))
loc=loc.drop_duplicates(ignore_index=True)

print(len(loc))

2874
5375
14419
22668
20469


In [8]:
loc.head()

,formatted_address,name,rating,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,big_list_name,adelaide?,south_australia?,sa?,australia?
0,"2-4 Kalimna Rd, Nuriootpa SA 5355, Australia",Western Ridge Brewing,4.9,-34.458271,138.998596,-34.456562,139.000131,-34.459262,138.997431,False,western ridge brewery,-1,-1,26,35
1,"149 Murray Ave, Renmark SA 5341, Australia",Renmark Irrigation Trust,0.0,-34.173396,140.750167,-34.172062,140.751637,-34.174761,140.748937,True,the renmark irrigation trust,-1,-1,24,33
2,"ThincLab, 10 Pulteney St, Adelaide SA 5000, Australia",Ortomi,0.0,-34.921765,138.605899,-34.920426,138.607052,-34.923126,138.604352,True,ortomi,26,-1,35,44
3,"10 Pulteney St Ground Floor (ThincLab, Adelaide SA 5000, Australia",AMTD Electronics,0.0,-34.921765,138.605899,-34.920518,138.607257,-34.923218,138.604557,True,ortomi,39,-1,48,57
4,"KarolinskÃ¡ 661/4, 186 00 RohanskÃ½ ostrov, Czechia",Photon Energy,4.0,50.094332,14.446315,50.095455,14.447729,50.092755,14.445030,False,equinor photon energy,-1,-1,-1,-1


In [9]:
df1.tech.unique()

array(['start-up', 'artificial intelligence', 'satellite technology',
       'energy storage technology', 'smart technology',
       'smart home technology', 'blockchain', 'VR technology',
       'digital health', 'data analytics', 'crowd funding',
       'cloud-based software', 'data storage', 'neuroplasticity',
       'aerospace engineering', 'renewable energy',
       'autonomous vehicle technology', 'automation systems',
       'space exploration', 'agtech', 'blockchain innovation',
       'smart technologies', 'photovoltaic panels', 'aerial-mapping',
       'virtual reality', 'augmented reality', 'e-invoicing', 'robotics',
       'point of sale'], dtype=object)

In [10]:
Aerospace_Space = [
    'satellite technology', 'aerospace engineering', 'space exploration'
]
Advanced_manufacturing = [
    'smart technology', 'smart home technology', 'automation systems',
    'smart technologies', 'photovoltaic panels', 'robotics'
]
Agtech = ['agtech']
Digital_infrastructure = [
    'blockchain', 'crowd funding', 'cloud-based software', 'data storage',
    'blockchain innovation'
]
Health_medical = ['digital health', 'neuroplasticity']
Machine_learning = [
    'data analytics',
    'autonomous vehicle technology',
    'automation systems',
    'blockchain innovation',
    'aerial-mapping',
    'e-invoicing',
    'robotics',
    'point of sale',
    'artificial intelligence',
    'smart technology',
    'smart home technology',
    'smart technologies',
]
Renewable_energy = ['energy storage technology', 'renewable energy']
VFX_VR_AR = ['VR technology', 'virtual reality', 'augmented reality']
start_up = ['start-up']

In [16]:
#prepare
groupdate = df1[(~df1['organization'].isna())
               & (df1['tech'].isin(Aerospace_Space))].groupby(
                   'date_')['organization'].apply(','.join).reset_index()
groupdate = groupdate.drop(axis=1, index=len(groupdate) - 1)

groupdate['year'] = groupdate['date_'].apply(
    lambda x: x.split(',')[1].strip(" "))

groupdate2 = groupdate.groupby('year')['organization'].apply(
    ','.join).reset_index()

In [25]:
sa_only = loc.reset_index(drop=True)
sa_only = sa_only[['geometry.location.lat','geometry.location.lng','big_list_name']]
sa_only = sa_only.reset_index()
sa_only = sa_only[~sa_only['big_list_name'].isna()].reset_index(drop=True)

#get the frame for the map
new = pd.DataFrame()

for i in range(len(sa_only)):
    try:
        temp = groupdate2['organization'].apply(lambda x: x.count(sa_only['big_list_name'].loc[i]))

        temp1 = groupdate2[['year']].copy()
        temp1['weight'] = temp
        temp1['lat'] = sa_only['geometry.location.lat'].loc[i]
        temp1['lng'] = sa_only['geometry.location.lng'].loc[i]
        temp1['saindex'] = sa_only['index'].iloc[i]

        new = pd.concat([new,temp1],ignore_index=True)
    except:
        print(i)
    
#get the data for time-lapse heat map
list_ = []
year_ = []

new['weight_2'] = winsorize((new['weight']), limits=[0, 0.005]).tolist()

new = new[new.weight_2!=0].reset_index(drop=True).copy()
new['weight_2'] = new['weight_2'] /new['weight_2'].abs().max()
# new['month'] = new.month_year.apply(lambda x: int(x.split('-')[0]))
# new['year'] = new.month_year.apply(lambda x: int(x.split('-')[1]))
new = new.sort_values(['year','month']).reset_index(drop=True)

AttributeError: 'DataFrame' object has no attribute 'month_year'

In [28]:
new = new.sort_values(['year']).reset_index(drop=True)

In [29]:
new

,year,weight,lat,lng,saindex,weight_2
0,2019,1,47.627065,-122.151530,15150,0.023256
1,2019,20,47.652468,-122.358257,11609,0.465116
2,2019,1,-42.882027,147.331852,4440,0.023256
3,2019,2,-36.313137,142.361338,4434,0.046512
4,2019,1,47.661172,-122.313981,4426,0.023256
...,...,...,...,...,...,...
8999,2022,1,51.046505,-114.070823,10306,0.023256
9000,2022,1,45.622988,-122.642494,10358,0.023256
9001,2022,5,47.610152,-122.332363,10361,0.116279
9002,2022,7,47.773738,-122.148799,10178,0.162791


In [ ]:
#prepare
groupdate = df1[~df1['organization'].isna()].groupby('date_')['organization'].apply(','.join).reset_index()
groupdate = groupdate.drop(axis=1,index=len(groupdate)-1)

groupdate['month'] = groupdate['date_'].apply(lambda x: x.split(',')[0].split(" ")[0])
groupdate['year'] = groupdate['date_'].apply(lambda x: x.split(',')[1].strip(" "))

groupdate['month'] = groupdate['month'].replace(
    to_replace=[
        'Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov',
        'Oct', 'Sep'
    ],
    value=[4, 8, 12, 2, 1, 7, 6, 3, 5, 11, 10, 9])

groupdate['month_year'] = groupdate[['month','year']].apply(lambda x: str(x[0])+'-'+x[1], axis = 1)

groupdate2 = groupdate.groupby('month_year')['organization'].apply(','.join).reset_index()

#try to filter sa_only
# sa_only = loc[(loc['adelaide?']!=-1) | (loc['south_australia?']!=-1) | (loc['sa?']!=-1)].reset_index(drop=True)
sa_only = loc.reset_index(drop=True)
sa_only = sa_only[['geometry.location.lat','geometry.location.lng','big_list_name']]
sa_only = sa_only.reset_index()
sa_only = sa_only[~sa_only['big_list_name'].isna()].reset_index(drop=True)

list_, year_ = prepare_map(groupdate2, sa_only)

# #get the frame for the map
# new = pd.DataFrame()

# for i in range(len(sa_only)):
#     try:
#         temp = groupdate2['organization'].apply(lambda x: x.count(sa_only['big_list_name'].loc[i]))

#         temp1 = groupdate2[['month_year']].copy()
#         temp1['weight'] = temp
#         temp1['lat'] = sa_only['geometry.location.lat'].loc[i]
#         temp1['lng'] = sa_only['geometry.location.lng'].loc[i]
#         temp1['saindex'] = sa_only['index'].iloc[i]

#         new = pd.concat([new,temp1],ignore_index=True)
#     except:
#         print(i)
    
# #get the data for time-lapse heat map
# list_ = []
# year_ = []

# new['weight_2'] = winsorize((new['weight']), limits=[0, 0.005]).tolist()

# new1 = new[new.weight_2!=0].reset_index(drop=True).copy()
# new1['weight_2'] = new1['weight_2'] /new1['weight_2'].abs().max()
# new1['month'] = new1.month_year.apply(lambda x: int(x.split('-')[0]))
# new1['year'] = new1.month_year.apply(lambda x: int(x.split('-')[1]))
# new1 = new1.sort_values(['year','month']).reset_index(drop=True)

# for sub in new1.groupby(['year','month']):
    
#     list_.append(sub[1][['lat','lng','weight_2']].values.tolist())
#     year_.append(str(sub[0][1])+'-'+str(sub[0][0]))

In [ ]:
#if we need the state bounderies
# lga_gdf = gpd.read_file(
#     r'C:\Users\duchi\OneDrive - University of South Australia\casual contract\6 Vij - 12.22\0 geo data\from Ali\SUA_2021_AUST_GDA2020.shp'
# )

# lga_gdf['boundary'] = lga_gdf.boundary
# # lga_gdf.head(1)

# # lga_gdf.plot(figsize=(20,20))
# # plt.show()
# lga_gdf['SUA_NAME21'].unique()
# lga_gdf = lga_gdf[(lga_gdf['SUA_NAME21'] =='Adelaide')]
# # lga_gdf['SUA_NAME21'].unique()

# # create map
# m = folium.Map([-25.714884, 135.219674],
#                zoom_start=4,
#                width=1000,
#                height=1000,
#                tiles=None)

# folium.TileLayer('CartoDB positron', name="Light Map", control=False).add_to(m)
# # list_orange_places = question2.city.unique().tolist()

# # for _, r in lga_gdf.iterrows():
# #     # Without simplifying the representation of each borough,
# #     # the map might not be displayed
# #     sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
# #     geo_j = sim_geo.to_json()
# #     geo_j = folium.GeoJson(data=geo_j)
# #     folium.Popup(r['SUA_NAME21']).add_to(geo_j)
# #     geo_j.add_to(m)

In [ ]:
# # location of all

# m = folium.Map([-25.714884, 135.219674], tiles=None, zoom_start=6)

# folium.TileLayer('CartoDB positron', name="Light Map", control=False).add_to(m)

# # hm = plugins.HeatMapWithTime(list_, index= year_, radius=6)

# # hm.add_to(m)

# # m

# for index, row in sa_only.iterrows():

#     folium.Marker([row['geometry.location.lat'], row['geometry.location.lng']],
#                   popup=row['big_list_name'],
#                   color='crimson',
#                   fill=True).add_to(m)
    
# m

In [ ]:
#final time-lapse heatmap
m = folium.Map([-25.714884, 135.219674], tiles=None, zoom_start=6)

folium.TileLayer('CartoDB positron', name="Light Map", control=False).add_to(m)

# hm = plugins.HeatMapWithTime(list_, index= year_, radius=5, blur=0.2, min_opacity=0.2)
hm = plugins.HeatMapWithTime(list_, index= year_, radius=5, blur=0.2, min_opacity=0.5 , use_local_extrema=False)

hm.add_to(m)


m

In [ ]:
# m.save('tech_heatmap.html')

In [ ]:
# Do the same thing bet for each group of tech

sa_only = loc.reset_index(drop=True)
sa_only = sa_only[['geometry.location.lat','geometry.location.lng','big_list_name']]
sa_only = sa_only.reset_index()
sa_only = sa_only[~sa_only['big_list_name'].isna()].reset_index(drop=True)
a = 0.005

list_1, year_1 = prepare_map(create_groupdate(df1, Aerospace_Space), sa_only, a)
list_2, year_2 = prepare_map(create_groupdate(df1, Advanced_manufacturing), sa_only, a)
list_3, year_3 = prepare_map(create_groupdate(df1, Agtech), sa_only, a)
list_4, year_4 = prepare_map(create_groupdate(df1, Digital_infrastructure), sa_only, a)
list_5, year_5 = prepare_map(create_groupdate(df1, Health_medical), sa_only, a)
list_6, year_6 = prepare_map(create_groupdate(df1, Machine_learning), sa_only, a)
list_7, year_7 = prepare_map(create_groupdate(df1, Renewable_energy), sa_only, a)
list_8, year_8 = prepare_map(create_groupdate(df1, VFX_VR_AR), sa_only, a)
list_9, year_9 = prepare_map(create_groupdate(df1, start_up), sa_only, a)

In [ ]:
test1 = create_groupdate(df1, Aerospace_Space)

In [ ]:
from jinja2 import Template 

from folium.map import Layer

class HeatMapWithTimeAdditional(Layer):
    _template = Template("""
        {% macro script(this, kwargs) %}
            var {{this.get_name()}} = new TDHeatmap({{ this.data }},
                {heatmapOptions: {
                    radius: {{this.radius}},
                    minOpacity: {{this.min_opacity}},
                    maxOpacity: {{this.max_opacity}},
                    scaleRadius: {{this.scale_radius}},
                    useLocalExtrema: {{this.use_local_extrema}},
                    defaultWeight: 1,
                    {% if this.gradient %}gradient: {{ this.gradient }}{% endif %}
                }
            }).addTo({{ this._parent.get_name() }});
        {% endmacro %}
    """)

    def __init__(self, data, name=None, radius=15,
                 min_opacity=0, max_opacity=0.6,
                 scale_radius=False, gradient=None, use_local_extrema=False,
                 overlay=True, control=True, show=True):
        super(HeatMapWithTimeAdditional, self).__init__(
            name=name, overlay=overlay, control=control, show=show
        )
        self._name = 'HeatMap'
        self.data = data

        # Heatmap settings.
        self.radius = radius
        self.min_opacity = min_opacity
        self.max_opacity = max_opacity
        self.scale_radius = 'true' if scale_radius else 'false'
        self.use_local_extrema = 'true' if use_local_extrema else 'false'
        self.gradient = gradient
        
# folium.plugins.HeatMapWithTime(list_1,overlay=False,name='Aerospace Space',index=year_1,min_opacity=0.05,radius=5).add_to(m)
# HeatMapWithTimeAdditional(list_2,overlay=False,name='Advanced manufacturing',min_opacity=0.05,radius=5).add_to(m)       

In [ ]:
m = folium.Map([-25.714884, 135.219674], tiles=None, zoom_start=6)

folium.TileLayer('CartoDB positron', name="Light Map", control=False).add_to(m)

# hm = plugins.HeatMapWithTime(list_, index= year_, radius=5, blur=0.2, min_opacity=0.2)
plugins.HeatMapWithTime(list_1, index= year_1, radius=5, blur=0.2, min_opacity=0.5 , use_local_extrema=False).add_to(m)
plugins.HeatMapWithTime(list_2, index= year_2, radius=5, blur=0.2, min_opacity=0.5 , use_local_extrema=False).add_to(m)

folium.LayerControl().add_to(m)

m